# Optimize - The Synthesis Engine

In the previous chapter we focused on how to design quantum algorithms with the Qmod language. Once a quantum algorithm is designed, it can be compiled into a quantum circuit implementation that can be executed on quantum computers or simulators. This is done by the Classiq synthesis engine. In this chapter we cover how to use and utilize the Classiq synthesis engine in the IDE and the Python SDK through concrete examples.

## Synthesis Background

In most cases, a specific quantum model can be compiled to many, sometime infinite number of quantum circuits that might differ in their properties. Some will have more qubits with smaller circuit depth, some will have all the qubits connected to each other whilst other will not, and some will have a fewer number of 2-qubit gates than others. 

The Classiq synthesis engine receives the quantum model as an input, together with the constraints and preferences of the desired quantum program, and outputs a quantum program implementation of the quantum model that satisfies the constraints and preferences.

Some of the current available constraints options for the Synthesis engine are:
* Optimization parameter - either to optimize for circuit width or circuit depth;
* Maximal gate count - maximum allowed number of a specific 1- or 2-qubit gate;
* Maximal circuit width or circuit depth.

And some of the available preferences:
* Compiling the quantum circuit for a specific quantum processor;
* The desired connectivity map of the quantum circuit;
* The output format of the quantum circuit, e.g. `QASM` or `QIR`.

A full list of possible [constraints](https://docs.classiq.io/latest/user-guide/platform/synthesis/constraints/) and [preferences](https://docs.classiq.io/latest/user-guide/platform/synthesis/preferences/) is available in the reference manual. 

In the following we cover how to apply constraints and preferences through a concrete example.

## Concrete Example

Let's go back to the example covered in the 'Design - The Qmod language' tutorial, where the task is to create a quantum algorithm that calculates in a superposition the arithmetic expression $y=x^2+1$. 

The following model written in Qmod implements the desired task:

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum], y: Output[QNum]):

    allocate(4, x)
    hadamard_transform(x)  # creates a uniform superposition
    y |= x**2 + 1

Which one can always directly synthesize without any constraints or preferences:

In [2]:
quantum_program = synthesize(create_model(main))

<div  style="text-align:center;">
    <img src="https://classiq-docs-images.s3.amazonaws.com/design.gif" style="width:100%;">
</div>

But in the following we cover how to do apply constraints and preferences. We first cover how to apply these in the IDE and then in the Python SDK.

## Constraints and Preferences in the IDE

In the IDE, once your model is completed, you can directly synthesize your algorithm with the default constraints and preferences by pressing the `Synthesize` button:


<div  style="text-align:center;" >
    <img src="https://classiq-docs-images.s3.amazonaws.com/optimize_default.png" style="width:100%;">
</div>

However, if you do want to apply some constraints and preferences, this can be done easily by adapting the parameters on the right-hand side of the screen and then synthesizing your model:

<div  style="text-align:center;" >
    <img src="https://classiq-docs-images.s3.amazonaws.com/optimize_adapt.png" style="width:100%;">
</div>

All the constraints and preferences that are available in the IDE can also be configured in the Python SDK as we cover next.

## Constraints and Preferences in the Python SDK 

The synthesis engine receives an underlying Qmod representation of the quantum model that is constructed in the Python SDK using the `create_model` function:

In [3]:
quantum_model = create_model(main)

This `quantum_model` can be directly synthesized with the command `synthesize(quantum_model)` to return the quantum program implementation. However, if we want to apply some constraints and preferences we need to adapt the `quantum_model` representation.

First, assume that we want to receive a circuit implementation with minimal number of qubits and with maximal circuit depth of $500$. So we can apply these as constraints to our `quantum_model`:

In [4]:
quantum_model_with_constraints = set_constraints(
    quantum_model, Constraints(optimization_parameter="width", max_depth=500)
)

Then the `quantum_model` can be then synthesized as usual:

In [5]:
quantum_program = synthesize(quantum_model_with_constraints)

And the parameters of the circuit implementation can be extracted:

In [6]:
circuit_width = QuantumProgram.from_qprog(quantum_program).data.width
circuit_depth = QuantumProgram.from_qprog(quantum_program).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")

The circuit width is 12 and the circuit_depth is 392


<details>
<summary> Compilation vs. Transpilation </summary>
The synthesis engine is a compiler that compiles a high-level functional model to one specific circuit out of many possible implementations. A transpiler on the other hand, transforms one circuit implementation to another one. Its use can be to change from a circuit representation with a given basis gate set to another one, or to further optimize a given circuit implementation with basic optimization procedures like cancellation of two identical Hermitian gates applied consequently.  
</details>

It is **highly recommended** to complete the following exercise in order to experience by yourself for the first time how the same quantum algorithm can be compiled into two different circuit implementation that can substantially differ from each other.

<details>
<summary> Recommended Exercise </summary>
Modify the constraints above such that you optimize the circuit for minimal circuit depth using maximum 25 qubits. What are the circuit depth and width you receive? Are they different than the above example? Try to analyze the two quantum circuits using `show(quantum_program)` and figure our which functional building block is implemented differently. 
</details>



Assume now that we want to execute the above quantum model on the IBM quantum processor `ibm_brisbane`. It is recommended to pass this information to the synthesis engine so the quantum program will be most adequate for the desired quantum computer. This is done by adding the following preference and re-synthesizing `quantum_model`:

In [7]:
quantum_model_with_preferences = set_preferences(
    quantum_model,
    Preferences(backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"),
)

quantum_program = synthesize(quantum_model_with_preferences)

<details>
<summary> Optional Exercise </summary>
Extract the circuit depth and width of the above quantum program. How do these differ from the previous values? Try to think why they differ in such a way. It is good to know that the `IBM Brisbane` device as specific limited connectivity between its qubits, so there might be a certain overhead in applying some 2-qubit gates...
</details>

So now you have a quantum program that implements the quantum model that calculates $y=x^2+1$ in a superposition, that is optimized for a specific real quantum computer. It only remains to follow the next chapters and to see how to actually run it on that computer with Classiq. But first we want to deep dive to the Analysis capabilities of Classiq.

In [8]:
write_qmod(quantum_model_with_constraints, "optimize")